In [2]:
!pip install langchain langchain-openai openai wikipedia -q
import os
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang("es")

# Configuración del LLM
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("GITHUB_BASE_URL"),
        openai_api_key=os.environ.get("GITHUB_TOKEN"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

✅ LLM de LangChain configurado.


In [3]:
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain import hub

@tool
def saludar_usuario(query: str) -> str:
    """Saludar amablemente al usuairo por su nombre."""
    
    try:
        return f"¡Hola {query}! ¿Cómo estás hoy?"
    except Exception as e:
        return f"Ocurrió un error: {e}"

tools = [saludar_usuario]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ Agente y herramientas listos.")

✅ Agente y herramientas listos.


In [4]:
from langchain.memory import ConversationSummaryMemory

# Esta memoria necesita un LLM para hacer los resúmenes
memory_summary = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

def chat_with_agent_summary(query: str):
    history = memory_summary.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({"input": query, "chat_history": history})
    memory_summary.save_context({"input": query}, {"output": response["output"]})
    return response["output"]

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_16944\1927192588.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_summary = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [5]:
print("Pregunta 1: ¿cuanto es 147 + 443?")
chat_with_agent_summary("cuanto es 147 + 443??")

print("Pregunta 2:  ¿cuanto es 915-458?")
chat_with_agent_summary("¿cuanto es 915-458?")

print("Pregunta 3: ¿Sobre qué estábamos hablando?")
response3_summary = chat_with_agent_summary("¿Sobre qué estábamos hablando?")
print(f"Respuesta 3: {response3_summary}")

Pregunta 1: ¿cuanto es 147 + 443?


> Entering new AgentExecutor chain...
147 + 443 es igual a **590**.

> Finished chain.
Pregunta 2:  ¿cuanto es 915-458?


> Entering new AgentExecutor chain...
915 - 458 es igual a **457**.

> Finished chain.
Pregunta 3: ¿Sobre qué estábamos hablando?


> Entering new AgentExecutor chain...
Estábamos hablando de cálculos matemáticos. Primero mencionaste una suma (147 + 443 = 590) y luego una resta en español (915 - 458 = 457). ¿Hay algo más en lo que te pueda ayudar?

> Finished chain.
Respuesta 3: Estábamos hablando de cálculos matemáticos. Primero mencionaste una suma (147 + 443 = 590) y luego una resta en español (915 - 458 = 457). ¿Hay algo más en lo que te pueda ayudar?


In [ ]:
@tool
def saludar(nombre: str) -> str:
    """Devuelve un saludo personalizado."""
    return f"¡Hola {nombre}! Encantado de conversar contigo."

tools = [saludar]


prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

def chat_with_agent(query: str):
    history = memory.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({
        "input": query,
        "chat_history": history
    })
    memory.save_context({"input": query}, {"output": response["output"]})
    return response["output"]


print("\n Chatbot iniciado (escribe 'salir' para terminar)\n")
while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit", "quit"]:
        print("¡Hasta luego!")
        break
    respuesta = chat_with_agent(user_input)
    print(f"Bot: {respuesta}\n")


🤖 Chatbot iniciado (escribe 'salir' para terminar)

Bot: 2 + 7 es igual a 9.

Bot: 9 * 7 es igual a 63.

Bot: Estamos hablando de matemáticas, específicamente de operaciones básicas como la suma y la multiplicación. ¿Hay algo más en lo que pueda ayudarte? 😊

Bot: ¿En qué puedo ayudarte hoy?

👋 ¡Hasta luego!
